In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import importlib
import data_cleaning as dc
%matplotlib inline


Tables of interest:
* imdb.title.basics  (for genres)
* imdb.title.ratings (for average ratings)
* bom.movie_gross (for studios)
* tn.movie_budgets (for budgets and worldwide gross)

## Import tables of interest

function `df_from_movie_csv` loads csv.gz by tablename into dataframe, setting index, converting date fields to date time, converting dollar fields to numeric.

In [121]:

import_tables = ['imdb.title.basics','imdb.title.ratings','bom.movie_gross','tn.movie_budgets']
dfs = {}
for table_name in import_tables:
    dfs[table_name] = dc.df_from_movie_csv(table_name)

function `clean_movie_df` standardizes field names, adds a `year` column if appropriate, and filters results to 2010-2018.

In [156]:
importlib.reload(dc)
for table_name in import_tables:
    dfs[table_name] = dc.clean_movie_df(dfs[table_name],table_name)

## Join tables

### IMDB

Join the two `imdb` dataframes. They join directly on index, no error-checking required.

In [157]:
imdb_title_ratings_df = dfs['imdb.title.basics'].join(dfs['imdb.title.ratings'])

In [84]:
imdb_title_ratings_df.size

1023008

In [158]:
imdb_title_ratings_df.head()

,title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


There are a lot of movies with the same `primary_title`. Dedupe by restricting to the entry with the highest `numvotes`.

In [159]:
imdb_title_ratings_df = dc.select_max_rows_on_key_column(imdb_title_ratings_df, 
                                                         key_column='title', 
                                                         max_column='numvotes')

In [160]:
imdb_title_ratings_df.shape[0]

136071

### Box Office Mojo

In [161]:
bom_df = dfs['bom.movie_gross']
bom_df = dc.select_max_rows_on_key_column(bom_df,key_column='title',max_column='domestic_gross')
bom_df.shape[0]

3386

Join the `imdb` dataframe with the `bom` dataframe on (English-language) movie title. This misses some big movies (title conventions are not identical).

In [162]:
imdb_bom_df = dc.join_dfs_on_key_col(imdb_title_ratings_df,bom_df,on='title')
imdb_bom_df.shape[0]

2605

Our matching by title has a success rate of 2605/3386 (77%).

### The Numbers

In [163]:
tn_df = dfs['tn.movie_budgets']
tn_df.head()

,release_date,title,production_budget,domestic_gross,worldwide_gross,year
id,,,,,,
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410.6,241.063875,1045.663875,2011
4,2015-05-01,Avengers: Age of Ultron,330.6,459.005868,1403.013963,2015
5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317.0,620.181382,1316.721747,2017
6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306.0,936.662225,2053.311220,2015
7,2018-04-27,Avengers: Infinity War,300.0,678.815482,2048.134200,2018


Join the `imdb_bom` dataframe with the `tn` dataframe on title and release year. 

In [164]:
df = dc.join_dfs_on_key_col(imdb_bom_df.drop(columns=['domestic_gross','foreign_gross']),tn_df,on=['title','year'])

In [165]:
df.shape[0]

1135

In [166]:
df.head()

,title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,studio,year,release_date,production_budget,domestic_gross,worldwide_gross
tconst,,,,,,,,,,,,,
tt1179933,10 Cloverfield Lane,10 Cloverfield Lane,2016,103.0,"Drama,Horror,Mystery",7.2,260383.0,Par.,2016,2016-03-11,5.0,72.082999,108.286422
tt1413492,12 Strong,12 Strong,2018,130.0,"Action,Drama,History",6.6,50155.0,WB,2018,2018-01-19,35.0,45.819713,71.118378
tt2024544,12 Years a Slave,12 Years a Slave,2013,134.0,"Biography,Drama,History",8.1,577301.0,FoxS,2013,2013-10-18,20.0,56.671993,181.025343
tt1542344,127 Hours,127 Hours,2010,94.0,"Adventure,Biography,Drama",7.6,323949.0,FoxS,2010,2010-11-05,18.0,18.335230,60.217171
tt2059171,13 Sins,13 Sins,2014,93.0,"Horror,Thriller",6.3,29550.0,RTWC,2014,2014-04-18,4.0,0.009134,0.047552


In [167]:
df['profit'] = df.worldwide_gross - df.production_budget

In [168]:
df['roi'] = df.profit / df.production_budget

In [169]:
df.head()

,title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,studio,year,release_date,production_budget,domestic_gross,worldwide_gross,profit,roi
tconst,,,,,,,,,,,,,,,
tt1179933,10 Cloverfield Lane,10 Cloverfield Lane,2016,103.0,"Drama,Horror,Mystery",7.2,260383.0,Par.,2016,2016-03-11,5.0,72.082999,108.286422,103.286422,20.657284
tt1413492,12 Strong,12 Strong,2018,130.0,"Action,Drama,History",6.6,50155.0,WB,2018,2018-01-19,35.0,45.819713,71.118378,36.118378,1.031954
tt2024544,12 Years a Slave,12 Years a Slave,2013,134.0,"Biography,Drama,History",8.1,577301.0,FoxS,2013,2013-10-18,20.0,56.671993,181.025343,161.025343,8.051267
tt1542344,127 Hours,127 Hours,2010,94.0,"Adventure,Biography,Drama",7.6,323949.0,FoxS,2010,2010-11-05,18.0,18.335230,60.217171,42.217171,2.345398
tt2059171,13 Sins,13 Sins,2014,93.0,"Horror,Thriller",6.3,29550.0,RTWC,2014,2014-04-18,4.0,0.009134,0.047552,-3.952448,-0.988112


In [170]:
df.describe()

,start_year,runtime_minutes,averagerating,numvotes,year,production_budget,domestic_gross,worldwide_gross,profit,roi
count,1135.000000,1135.000000,1134.000000,1.134000e+03,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000
mean,2013.579736,109.335683,6.472928,1.387685e+05,2013.663436,49.292330,64.145831,161.681521,112.389190,3.514366
std,2.511092,17.613655,0.930022,1.729823e+05,2.509142,57.440479,87.358082,243.021924,201.431578,13.954478
min,2010.000000,41.000000,1.600000,1.000000e+01,2010.000000,0.050000,0.000000,0.001242,-110.450242,-0.999862
25%,2011.000000,97.000000,5.900000,3.483400e+04,2011.000000,11.000000,9.934055,20.968331,3.920029,0.240090
50%,2013.000000,106.000000,6.500000,8.087400e+04,2014.000000,28.000000,35.593113,70.536870,36.118378,1.493783
75%,2016.000000,119.000000,7.100000,1.698302e+05,2016.000000,60.000000,79.145948,186.798798,124.707235,3.405503
max,2018.000000,180.000000,8.800000,1.841066e+06,2018.000000,410.600000,700.059566,2048.134200,1748.134200,415.564740
